In [1]:
import casadi as ca
import os

In [2]:
nx, ny = 3, 2

x_sym  = ca.SX.sym('x', nx)      # [n(1), v(1), a(1)]
dt_sym = ca.SX.sym('dt')         # scalar

# Unpack
q     = x_sym[0]               # joint position
dq     = x_sym[1]              # joint velocity
ddq     = x_sym[2]             # joint acceleration

# Discrete dynamics, constant acceleration over the step
q_next = q + dq*dt_sym + 0.5*ddq*(dt_sym**2)
dq_next = dq + ddq*dt_sym
ddq_next = ddq                        # zero jerk model

x_pred = ca.vertcat(q_next, dq_next, ddq_next)


# State Prediction and State Jacobian A = ∂f/∂x for EKF linearization
AF  = ca.jacobian(x_pred, x_sym)

In [3]:
# Define the measurement function.
# the measurements are: 
# [q_meas, dq_meas] 
h_expr = ca.vertcat(
    q,        # joint encoder position
    dq,        # joint velocity
)

# Compute the measurement Jacobian: H = ∂h/∂x
H_sym = ca.jacobian(h_expr, x_sym)

# a function for the measurement model and its Jacobian
h_func = ca.Function('h_func', [x_sym], [h_expr, H_sym])

In [4]:
# --- EKF iteration as a Casadi function ---
y_sym = ca.SX.sym('y', ny)       # measurement vector
P_sym = ca.SX.sym('P', nx, nx)   # error covariance

# Process and measurement noise covariances
Q_sym = ca.SX.sym('Q', nx, nx)   # process noise covariance
R_sym = ca.SX.sym('R', ny, ny)   # measurement noise covariance

# Prediction step:
#   x_pred = f(x,u,dt)
#   P_pred = AF * P * AF^T + Q
P_pred = AF @ P_sym @ AF.T + Q_sym

# Measurement update:
#   Compute the predicted measurement and its Jacobian
h_pred, H = h_func(x_pred)

# Innovation (measurement residual)
Y_corrected_k = ca.diag([1,  1])@y_sym #no scaling or correction applied


y_err = Y_corrected_k - h_pred
# Innovation covariance
S = H @ P_pred @ H.T + R_sym
# Kalman gain
K = P_pred @ H.T @ ca.inv(S)
# Updated state estimate
x_upd = x_pred + K @ y_err
# Updated covariance
P_upd = (ca.SX.eye(nx) - K @ H) @ P_pred

# Create the EKF update function.
ekf_update = ca.Function('ekf_update',
                           [x_sym, P_sym, dt_sym, y_sym, Q_sym, R_sym],
                           [x_upd, P_upd, y_err],
                           ['x', 'P','dt', 'y', 'Q', 'R'],
                           ['x_next', 'P_next', 'y_err'])

In [5]:
# ekf_update.save('EKF_update.casadi')

# c , cpp or matlab code generation for forward dynamics
ekf_update.generate("EKFnext.c")
os.system(f"gcc -fPIC -shared EKFnext.c -o libmEKF_next.so")

0

In [6]:
ekf_update(x=ca.DM.ones(nx), P=ca.DM.eye(nx), dt=0.1, y=ca.DM.zeros(ny), Q=ca.DM.eye(nx)*0.1, R=ca.DM.eye(ny)*0.1)

{'P_next': DM(
 [[0.0916783, 0.000691182, -0.000275097], 
  [0.000691182, 0.0916781, 0.00828731], 
  [-0.000275097, 0.00828731, 1.09173]]),
 'x_next': DM([0.0843518, 0.083903, 0.911879]),
 'y_err': DM([-1.105, -1.1])}